In [1]:
import os
import torch
import importlib
module = importlib.import_module('transformers.models.qwen2.modeling_qwen2')
from transformers import AutoConfig
import time
from transformers.models.qwen2.modeling_qwen2 import *
from copy import deepcopy
from tqdm import tqdm

/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

config = AutoConfig.from_pretrained('/mnt/workspace/mdy/models/Qwen2.5-7B-Instruct')
dtype = torch.bfloat16
bs = 8
seq_len = 2048
iters = 512
old = Qwen2DecoderLayer(config, layer_idx=0).cuda().to(dtype)
x = torch.randn(bs, seq_len, config.hidden_size).cuda().to(dtype)
x.requires_grad_(True)
cos = torch.randn(bs, seq_len, config.hidden_size//config.num_attention_heads).cuda().to(dtype)
sin = torch.rand_like(cos)
dy = torch.randn_like(x)

In [ ]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    old_y = old(x, position_embeddings=(cos,sin))[0]
    old_y.backward(dy)
t1 = time.time() - t
t1

  0%|          | 0/512 [00:00<?, ?it/s]/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
 27%|██▋       | 137/512 [00:21<01:00,  6.18it/s]

In [3]:
if x.grad is not None:
    x.grad.zero_()
old_y = old(x, position_embeddings=(cos,sin))[0]
old_y.backward(dy)
old_grad = deepcopy(x.grad)


/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [3]:
from replace_qwen2_kernel import trigger

/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [4]:
if x.grad is not None:
    x.grad.zero_()
new_y = old(x, position_embeddings=(cos, sin))[0]
new_y.backward(dy)
new_grad = deepcopy(x.grad)

1


/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [ ]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    old_y = old(x, position_embeddings=(cos,sin))[0]
    old_y.backward(dy)
t2 = time.time() - t
t2

  0%|          | 0/512 [00:00<?, ?it/s]/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
 69%|██████▉   | 353/512 [00:51<00:23,  6.78it/s]

In [11]:
old_y[0]

tensor([[-1.1953, -0.4570, -0.5195,  ..., -0.8477,  0.4590, -0.1885],
        [-0.1123,  0.5000,  0.6367,  ...,  1.4531, -1.0469, -1.4531],
        [-0.4531,  0.7734, -0.5117,  ...,  0.1670,  0.0508, -0.7305],
        ...,
        [-0.2012,  0.1729,  1.1719,  ..., -1.0234, -0.8516, -1.0234],
        [ 0.6406,  0.0625, -0.0825,  ...,  0.5117,  0.1963,  0.2637],
        [-0.5742, -0.4512,  1.4219,  ..., -2.5938,  0.4336, -0.4844]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [12]:
new_y[0]

tensor([[-1.1953, -0.4570, -0.5195,  ..., -0.8477,  0.4590, -0.1875],
        [-0.1128,  0.5000,  0.6367,  ...,  1.4531, -1.0469, -1.4531],
        [-0.4531,  0.7734, -0.5117,  ...,  0.1680,  0.0503, -0.7305],
        ...,
        [-0.2012,  0.1729,  1.1719,  ..., -1.0234, -0.8516, -1.0234],
        [ 0.6406,  0.0623, -0.0825,  ...,  0.5117,  0.1953,  0.2637],
        [-0.5742, -0.4512,  1.4219,  ..., -2.5938,  0.4336, -0.4824]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [13]:
old_grad[0]

tensor([[ 1.2578, -0.6133,  0.4766,  ...,  0.2285, -0.2812, -2.7969],
        [ 1.2344, -1.3203,  0.8867,  ...,  0.0474, -0.5078,  0.2852],
        [-1.7734, -0.6367,  1.3516,  ...,  0.7344,  0.1299,  2.0000],
        ...,
        [ 0.1465,  0.7031,  0.2988,  ..., -1.1484,  0.8594, -1.3750],
        [-0.7930, -1.7812, -0.7969,  ..., -0.2637,  0.8594,  0.6445],
        [ 2.1250,  0.6328, -0.7852,  ...,  0.0801,  1.8281,  1.2500]],
       device='cuda:0', dtype=torch.bfloat16)

In [14]:
new_grad[0]

tensor([[ 1.2578, -0.6055,  0.4688,  ...,  0.2256, -0.2852, -2.8125],
        [ 1.2422, -1.3203,  0.8750,  ...,  0.0505, -0.5039,  0.3008],
        [-1.7734, -0.6211,  1.3594,  ...,  0.7227,  0.1260,  1.9922],
        ...,
        [ 0.1592,  0.6992,  0.2891,  ..., -1.1484,  0.8555, -1.3828],
        [-0.7969, -1.7812, -0.8086,  ..., -0.2578,  0.8750,  0.6523],
        [ 2.1406,  0.6211, -0.7773,  ...,  0.0732,  1.8281,  1.2578]],
       device='cuda:0', dtype=torch.bfloat16)